In [38]:
import pandas as pd
import nltk
import re
from bs4 import BeautifulSoup
import numpy as np
import gensim 
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 
import logging

In [39]:
# Hide all the warnings in jupyter notebook

import warnings
warnings.filterwarnings ('ignore')

In [40]:
df1 = pd.read_csv('AppReviews.csv',encoding='utf-8')
df2 = pd.read_csv('JIRA.csv',encoding='utf-8')
df3 = pd.read_csv('StackOverflow.csv',encoding='utf-8').drop('id', axis=1)
df4 = pd. concat([df1, df2, df3])

In [41]:
df4['sentence'].to_csv('alice.txt', sep = "\n", index = False)

sample = open("alice.txt", "r",encoding="utf8") 
s = sample.read() 
f = s.replace("\n", " ") 

In [42]:
data = [] 
for i in sent_tokenize(f): 
	temp = [] 
	
	# tokenize the sentence into words 
	for j in word_tokenize(i): 
		temp.append(j.lower()) 

	data.append(temp) 
model1 = gensim.models.Word2Vec(data, min_count = 1,vector_size=100, window = 5) 
#<---deprecated model1 = gensim.models.Word2Vec(data, min_count = 1,size=100, window = 5)-------->deprecated#

model2 = gensim.models.Word2Vec(data, min_count = 1,vector_size=100,window = 5, sg = 1) 

#<---deprecated model2 = gensim.models.Word2Vec(data, min_count = 1,size=100,window = 5, sg = 1-------->deprecated#

In [43]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS =nltk.corpus.stopwords.words('english')

In [44]:
def clean_text(text):
    if type(text)==str:
        
        text = BeautifulSoup(text, "lxml").text # HTML decoding
        text = text.lower() # lowercase text
        text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
        text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
        text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
        return text
    return ""

In [45]:
def wordprint(words,model1):
    all_words, mean = set(), []
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in model1.wv.key_to_index:
            a=np.array(model1.wv[word])
            mean.append(a)
    if not mean:
        # FIXME: remove these examples in pre-processing
        return np.zeros(100,)   
    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

In [46]:
def  word_averaging_listn(wv, text_list):
    return np.vstack([wordprint(post,wv) for post in text_list ])

In [47]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [48]:
import numpy as np
fn=['AppReviews','JIRA','StackOverflow']
for i in range(0,3):
    fname=fn[i]+'.csv'
    df = pd.read_csv(fname,encoding='utf-8')
    df['sentence'] = df['sentence'].apply(clean_text)
    print(df.head())
    comtdata=df
    test_tokenized = comtdata.apply(lambda r: w2v_tokenize_text(r['sentence']), axis=1).values
    X_comtdata_average1 = word_averaging_listn(model1,test_tokenized)
    fname=fn[i]+'cbow.csv'
    np.savetxt(fname,X_comtdata_average1, delimiter=',', fmt='%f')
    X_comtdata_average1 = word_averaging_listn(model2,test_tokenized)
    fname=fn[i]+'skg.csv'
    np.savetxt(fname,X_comtdata_average1, delimiter=',', fmt='%f')

                                            sentence  oracle
0  package file invalid phone factory reset wante...      -1
1  iffy nice clean app sometimes works times does...      -1
2                             cool freezes everytime      -1
3  network error suddenly downloading update pack...      -1
4  annoying let choose pictures want freezes forc...      -1
                                            sentence  oracle
0                                        guys stupid      -1
1  lost whole morning cause hbases regionserver d...      -1
2                       quote messing deep hbase dfs      -1
3                   think going sweep shit kill root      -1
4  idiot yeah idiotpath good commonslang hairball...      -1
    id                                           sentence  oracle
0    6                                      sadly working      -1
1   78  everything builds fine try deploy application ...      -1
2   90                     causing null pointer exception      -1
3  1